# STA 141B Lecture 5

The class website is <https://github.com/2019-winter-ucdavis-sta141b/notes>

### Announcements

* Assignment 2 will be posted later today
* Information about the project and groups later this week

### Topics

* NumPy Example
* Pandas

### Data Sets

* Dogs (in repository)

### References

* Python for Data Analysis, Ch. 5, 10
* [Python Data Science Handbook][PDSH], Ch. 3

[PDSH]: https://jakevdp.github.io/PythonDataScienceHandbook/
[ProGit]: https://git-scm.com/book/

## Example - Using NumPy for Monte Carlo Integration

Consider a circle with radius 1 circumscribed by a square with side length 2.

The area of the circle is $\pi$, so for a uniform distribution on the square, the probability a point will fall in the circle is $\pi / 4$.

We can estimate the probability to estimate $\pi$.

In [28]:
# Estimate pi


import numpy as np

def estimate_pi(n = 100_000):
    # First, we need to sample points (x, y) with x in (-1, 1) and y the same
    x = np.random.rand(n)
    y = np.random.rand(n)

    # Check which are in the circle
    in_circle = x**2 + y**2 <= 1

    # Estimate pi
    #sum(in_circle)
    num_inside = in_circle.sum()
    p_hat = num_inside / n
    return 4 * p_hat

estimate_pi(10)

3.2

## Pandas

Pandas is a Python package that provides tools for manipulating tabular data. The name "pandas" is short for "**PAN**el **DA**ta", an econometrics term. Since we're using Anaconda, Pandas is already installed.

Pandas is documented [here](http://pandas.pydata.org/pandas-docs/stable/).

In [29]:
import pandas as pd

### Series

A Pandas Series is a generalization of a NumPy array.

In addition to elements, every series includes an _index_.

In [31]:
x = pd.Series([1, 2, 3])
x

0    1
1    2
2    3
dtype: int64

A series can be indexed in all of the same ways as a NumPy array, but also by index values.

This means a series can also be used like an ordered dictionary.

In [33]:
x[0:2]

0    1
1    2
dtype: int64

In [36]:
x = pd.Series([1, 2, 3], index = ["a", "b", "c"])
x

a    1
b    2
c    3
dtype: int64

In [37]:
x["a"]

1

In [38]:
x[0]

1

In [39]:
x["a":"b"] # careful, this is inclusive!

a    1
b    2
dtype: int64

What if a series has integer indexes?

In [40]:
y = pd.Series([1, 2, 3], index = [10, 12, 14])
y

10    1
12    2
14    3
dtype: int64

In [41]:
y[0] # ambiguous

KeyError: 0

For a indexing series (and as we'll see later, also data frames):

* `[ ]` is by position, name, or condition. **Exception:** for an integer index it's by name or condition only.
* `.iloc[ ]` is by position
* `.loc[ ]` is by name or condition

In [42]:
y.iloc[0]

1

In [43]:
y.loc[10]

1

In [44]:
y[10]

1

In [45]:
z = pd.Series([1, 2, 3, 4], index = [3j, 2j, 1j, -1j])
z

3j         1
2j         2
1j         3
(-0-1j)    4
dtype: int64

In [46]:
z[3j]

1

In [47]:
z.index

Index([3j, 2j, 1j, (-0-1j)], dtype='object')

In [48]:
z.index = [0, 1, 2, 3]

In [49]:
z

0    1
1    2
2    3
3    4
dtype: int64

### Data Frames

A Pandas Data Frame represents tabular data as a collection of Series.

In [54]:
df = pd.DataFrame({"x": [1, 2, 3], "y": ["a", "b", "c"]})
df

,x,y
0,1,a
1,2,b
2,3,c


Data frames support the similar indexing methods as series. However, for indexing with `[ ]`,

* Scalar values get columns by name
* Conditions or slices get rows

In [55]:
df.iloc[0, 0]

1

In [56]:
df["x"]

0    1
1    2
2    3
Name: x, dtype: int64

In [57]:
df[0] # scalars always look up columns by name

KeyError: 0

In [58]:
df.loc[:, "x"]

0    1
1    2
2    3
Name: x, dtype: int64

In [60]:
df[df["x"] > 2]

,x,y
2,3,c


In [61]:
df.loc[df["x"] > 2, :]

,x,y
2,3,c


In [65]:
df.shape

(3, 3)

In [66]:
df.size

9

In [67]:
type(df)

pandas.core.frame.DataFrame

In [68]:
df.dtypes

x    float64
y      int64
z     object
dtype: object

### Missing Data

Pandas represents missing data with `NaN` and `None`, but these values do not exclusively mean missing data. For instance, `NaN` stands for "Not a Number" and is also the result of undefined computations. Pay attention to your data and code to determine whether values are missing or have some other meaning.

You can create `NaN` values with NumPy.

In [62]:
np.nan

nan

In [63]:
df = pd.DataFrame({"x": [1, np.nan, 2], "y": np.arange(3), "z": ["x", "y", None]})
df

,x,y,z
0,1.0,0,x
1,NaN,1,y
2,2.0,2,None


Use the `.isna()` or `.isnull()` methods to detect missing values.

In [64]:
df.isna()

,x,y,z
0,False,False,False
1,True,False,False
2,False,False,True


In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
x    2 non-null float64
y    3 non-null int64
z    2 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 152.0+ bytes


In [77]:
(~df.isna()).sum().sum() # number not missing

7

In [78]:
df.notna().sum().sum()

7

### Data Alignment

Pandas supports vectorized operations, but elements are automatically aligned by index. **Beware!!** This is a major difference compared to R.

In [83]:
x = pd.Series([1, 2, 3], index = ["a", "b", "c"])
y = pd.Series([1, 2, 3, 1], index = ["b", "a", "d", "c"])

x

a    1
b    2
c    3
dtype: int64

In [84]:
y

b    1
a    2
d    3
c    1
dtype: int64

In [85]:
x * y

a    2.0
b    2.0
c    3.0
d    NaN
dtype: float64

You can use the `.reset_index()` method to reset the indexes on a series or data frame.

In [88]:
x.reset_index() # keep old index as a new column

x.reset_index(drop = True) # throw away the old index

0    1
1    2
2    3
dtype: int64

### Reading Data

Pandas provides functions for reading (and writing) a variety of common formats. Most of their names begin with `read_`. For instance, we can read the dogs data from a CSV file:

In [89]:
dogs = pd.read_csv("dogs_full.csv")

In [90]:
dogs.head()

,breed,group,datadog,popularity_all,popularity,lifetime_cost,intelligence_rank,longevity,ailments,price,food_cost,grooming,kids,megarank_kids,megarank,size,weight,height
0,Border Collie,herding,3.64,45,39.0,20143.0,1.0,12.52,2.0,623.0,324.0,weekly,low,1.0,29.0,medium,NaN,20.0
1,Border Terrier,terrier,3.61,80,61.0,22638.0,30.0,14.00,0.0,833.0,324.0,weekly,high,2.0,1.0,small,13.5,NaN
2,Brittany,sporting,3.54,30,30.0,22589.0,19.0,12.92,0.0,618.0,466.0,weekly,medium,3.0,11.0,medium,35.0,19.0
3,Cairn Terrier,terrier,3.53,59,48.0,21992.0,35.0,13.84,2.0,435.0,324.0,weekly,high,4.0,2.0,small,14.0,10.0
4,Welsh Springer Spaniel,sporting,3.34,130,81.0,20224.0,31.0,12.49,1.0,750.0,324.0,weekly,high,5.0,4.0,medium,NaN,18.0


### Inspecting Data

Series and data frames provide many of the same methods and attributes as NumPy arrays.

For a data frame, the `.dtypes` attribute gives the column types.

The type "object" means some non-numeric Python object, often a string.

In [91]:
dogs.dtypes

breed                 object
group                 object
datadog              float64
popularity_all         int64
popularity           float64
lifetime_cost        float64
intelligence_rank    float64
longevity            float64
ailments             float64
price                float64
food_cost            float64
grooming              object
kids                  object
megarank_kids        float64
megarank             float64
size                  object
weight               float64
height               float64
dtype: object

There are also several methods for quickly summarizing data.

In [93]:
dogs.describe()

,datadog,popularity_all,popularity,lifetime_cost,intelligence_rank,longevity,ailments,price,food_cost,megarank_kids,megarank,weight,height
count,87.000000,172.000000,87.000000,91.000000,132.000000,135.000000,148.000000,146.000000,87.000000,87.000000,87.000000,86.00000,159.000000
mean,2.603678,87.122093,44.000000,19819.538462,40.924242,10.956741,1.216216,876.815068,489.597701,43.954023,43.942529,44.97093,19.089623
std,0.570288,50.205335,25.258662,3102.475382,19.603560,1.995742,1.549810,461.172524,204.266894,25.288065,25.278153,35.52707,6.012400
min,0.990000,1.000000,1.000000,12653.000000,1.000000,6.290000,0.000000,283.000000,270.000000,1.000000,1.000000,5.00000,5.000000
25%,2.185000,43.750000,22.500000,17816.500000,27.000000,9.700000,0.000000,587.250000,324.000000,22.500000,22.500000,17.50000,14.000000
50%,2.710000,87.500000,44.000000,20087.000000,42.000000,11.290000,1.000000,795.000000,466.000000,44.000000,44.000000,35.00000,19.000000
75%,3.035000,130.250000,65.500000,21798.000000,54.250000,12.365000,2.000000,1042.250000,466.000000,65.500000,65.500000,62.50000,24.125000
max,3.640000,173.000000,87.000000,26686.000000,80.000000,16.500000,9.000000,3460.000000,1349.000000,87.000000,87.000000,175.00000,32.000000


In [95]:
# First, get the string columns ("object"), then describe
dogs.select_dtypes(include = ["object"]).describe()

,breed,group,grooming,kids,size
count,172,172,112,112,172
unique,172,7,3,3,3
top,American English Coonhound,sporting,weekly,high,medium
freq,1,28,88,67,60


### Aggregation

Pandas also provides several methods for aggregating data, such as `.mean()`, `.median()`, `.std()`, and `.value_counts()`. They ignore missing values by default.

In [96]:
dogs.median()

datadog                  2.71
popularity_all          87.50
popularity              44.00
lifetime_cost        20087.00
intelligence_rank       42.00
longevity               11.29
ailments                 1.00
price                  795.00
food_cost              466.00
megarank_kids           44.00
megarank                44.00
weight                  35.00
height                  19.00
dtype: float64

In [97]:
dogs["price"].median()

795.0

In [99]:
dogs["group"].value_counts() # like R's table() with 1 arg

sporting        28
terrier         28
working         27
hound           26
herding         25
toy             19
non-sporting    19
Name: group, dtype: int64

For counting one group against another (crosstabulating), use `pd.crosstab()`.

In [100]:
pd.crosstab(dogs["group"], dogs["kids"]) # like R's table() with 2+ arg

kids,high,low,medium
group,,,
herding,11,2,3
hound,8,1,6
non-sporting,7,0,4
sporting,17,1,3
terrier,9,0,9
toy,3,5,5
working,12,1,5


### Applying Functions

You can also use Pandas to apply your own aggregation functions to columns or rows.

* `.apply()` applies a function column-by-column or row-by-row.
* `.applymap()` applies a function element-by-element.


In [103]:
def spread(x):
    # Input is a single column (or row)
    return x.max() - x.min()
    
dogs.select_dtypes(include = ["float64", "int64"]).apply(spread)

datadog                  2.65
popularity_all         172.00
popularity              86.00
lifetime_cost        14033.00
intelligence_rank       79.00
longevity               10.21
ailments                 9.00
price                 3177.00
food_cost             1079.00
megarank_kids           86.00
megarank                86.00
weight                 170.00
height                  27.00
dtype: float64

### Grouping

Use the `.groupby()` method to group data before computing aggregate statistics.

In [104]:
dogs.head()

,breed,group,datadog,popularity_all,popularity,lifetime_cost,intelligence_rank,longevity,ailments,price,food_cost,grooming,kids,megarank_kids,megarank,size,weight,height
0,Border Collie,herding,3.64,45,39.0,20143.0,1.0,12.52,2.0,623.0,324.0,weekly,low,1.0,29.0,medium,NaN,20.0
1,Border Terrier,terrier,3.61,80,61.0,22638.0,30.0,14.00,0.0,833.0,324.0,weekly,high,2.0,1.0,small,13.5,NaN
2,Brittany,sporting,3.54,30,30.0,22589.0,19.0,12.92,0.0,618.0,466.0,weekly,medium,3.0,11.0,medium,35.0,19.0
3,Cairn Terrier,terrier,3.53,59,48.0,21992.0,35.0,13.84,2.0,435.0,324.0,weekly,high,4.0,2.0,small,14.0,10.0
4,Welsh Springer Spaniel,sporting,3.34,130,81.0,20224.0,31.0,12.49,1.0,750.0,324.0,weekly,high,5.0,4.0,medium,NaN,18.0


In [106]:
dogs.groupby("group").mean()

,datadog,popularity_all,popularity,lifetime_cost,intelligence_rank,longevity,ailments,price,food_cost,megarank_kids,megarank,weight,height
group,,,,,,,,,,,,,
herding,2.732000,99.880000,43.500000,20691.818182,21.812500,11.728824,2.235294,814.941176,490.900000,40.300000,42.600000,36.666667,19.730000
hound,2.373077,104.769231,52.692308,19365.769231,54.904762,10.793529,0.833333,746.571429,514.538462,54.769231,56.153846,63.833333,22.543478
non-sporting,2.488000,82.210526,38.600000,19315.800000,46.714286,10.976000,1.352941,930.500000,409.200000,46.300000,42.800000,27.928571,14.984375
sporting,2.976000,87.428571,46.066667,20299.312500,27.782609,10.895600,1.040000,760.125000,510.866667,27.466667,17.266667,51.966667,21.276786
terrier,2.787500,100.250000,58.416667,20504.333333,44.750000,11.480000,0.653846,905.760000,389.916667,37.166667,39.583333,23.413043,13.780000
toy,2.805385,54.052632,36.769231,19506.076923,48.647059,11.672941,1.000000,686.894737,343.230769,35.076923,49.000000,9.818182,10.533333
working,2.065000,71.111111,32.285714,19164.687500,41.529412,9.465909,1.772727,1235.708333,721.500000,66.571429,62.000000,105.000000,25.388889


By default, the groups become the index. You can keep them as regular columns by setting `as_index = False` when grouping.

In [107]:
dogs.groupby("group", as_index = False).mean()

,group,datadog,popularity_all,popularity,lifetime_cost,intelligence_rank,longevity,ailments,price,food_cost,megarank_kids,megarank,weight,height
0,herding,2.732000,99.880000,43.500000,20691.818182,21.812500,11.728824,2.235294,814.941176,490.900000,40.300000,42.600000,36.666667,19.730000
1,hound,2.373077,104.769231,52.692308,19365.769231,54.904762,10.793529,0.833333,746.571429,514.538462,54.769231,56.153846,63.833333,22.543478
2,non-sporting,2.488000,82.210526,38.600000,19315.800000,46.714286,10.976000,1.352941,930.500000,409.200000,46.300000,42.800000,27.928571,14.984375
3,sporting,2.976000,87.428571,46.066667,20299.312500,27.782609,10.895600,1.040000,760.125000,510.866667,27.466667,17.266667,51.966667,21.276786
4,terrier,2.787500,100.250000,58.416667,20504.333333,44.750000,11.480000,0.653846,905.760000,389.916667,37.166667,39.583333,23.413043,13.780000
5,toy,2.805385,54.052632,36.769231,19506.076923,48.647059,11.672941,1.000000,686.894737,343.230769,35.076923,49.000000,9.818182,10.533333
6,working,2.065000,71.111111,32.285714,19164.687500,41.529412,9.465909,1.772727,1235.708333,721.500000,66.571429,62.000000,105.000000,25.388889


You can group by multiple columns.

In [108]:
dogs.groupby(["group", "kids"]).mean()

datadog  popularity_all  popularity  lifetime_cost  \
group        kids                                                          
herding      high    2.576667       96.000000   53.500000   21171.142857   
             low     3.445000       52.500000   44.000000   20269.000000   
             medium  2.485000       61.666667   13.000000   19437.000000   
hound        high    2.295000       98.250000   54.166667   20165.500000   
             low     3.190000        9.000000    9.000000   20113.000000   
             medium  2.315000       82.500000   58.500000   18441.500000   
non-sporting high    2.723333       70.714286   42.333333   20368.000000   
             medium  2.135000       42.000000   33.000000   17737.500000   
sporting     high    2.935714       70.176471   47.214286   20146.666667   
             low          NaN      161.000000         NaN            NaN   
             medium  3.540000      108.333333   30.000000   22589.000000   
terrier      high    3.024000       98.000000   57.200000   21680.600000   
             medium  2.618571       89.111111   59.285714   19664.142857   
toy          high    2.740000       19.333333   19.333333   19439.333333   
             low     2.844000       36.800000   30.800000   20409.800000   
             medium  2.806000       77.400000   53.200000   18642.400000   
working      high    1.992222       53.333333   30.111111   18153.272727   
             low     1.950000       47.000000   41.000000   20994.000000   
             medium  2.257500       57.400000   35.000000   21488.750000   

                     intelligence_rank  longevity  ailments        price  \
group        kids                                                          
herding      high            26.125000  11.702000  2.111111   848.875000   
             low              5.500000  12.095000  1.500000   576.500000   
             medium          22.500000  10.936667  5.000000   692.500000   
hound        high            56.625000  10.678571  1.000000  1028.714286   
             low             49.000000  12.630000  2.000000   423.000000   
             medium          61.666667  10.535000  1.166667   841.000000   
non-sporting high            50.428571  11.844286  0.571429   896.333333   
             medium          54.750000   9.890000  2.250000  1308.750000   
sporting     high            24.125000  10.986250  1.470588   705.875000   
             low                   NaN  11.170000  0.000000          NaN   
             medium          31.333333  11.190000  0.000000   618.000000   
terrier      high            48.875000  12.213333  0.750000   800.000000   
             medium          43.250000  11.391429  0.888889  1042.555556   
toy          high            57.000000  11.830000  1.333333   689.666667   
             low             54.400000  12.674000  1.200000   785.600000   
             medium          37.400000  11.150000  1.400000   746.000000   
working      high            48.444444   8.947273  2.300000  1292.083333   
             low             54.000000  10.160000  1.000000  1202.000000   
             medium          23.000000  10.172000  2.500000   982.000000   

                      food_cost  megarank_kids   megarank      weight  \
group        kids                                                       
herding      high    531.166667      48.833333  37.500000   31.000000   
             low     395.000000       5.000000  40.500000         NaN   
             medium  466.000000      50.000000  60.000000         NaN   
hound        high    601.500000      60.666667  49.166667   72.500000   
             low     324.000000      14.000000  54.000000   24.000000   
             medium  459.333333      55.666667  63.500000   67.000000   
non-sporting high    395.000000      39.000000  27.666667   18.000000   
             medium  430.500000      57.250000  65.500000   34.500000   
sporting     high    514.071429      29.214286  17.714286   55.791667   
             low         

On groups, the `.apply()` method computes group-by-group. It is the most general form of two other methods:

* `.agg()`, which applies a function to each group to compute summary statistics
* `.transform()`, which applies a function to each group to compute transformations (such as standardization)

In [ ]:
def f(x):
    # Like R's browser() function, the breakpoint() function runs the debugger.
    # The debugger is especially useful for checking what's going on in a function.
    breakpoint()
    
dogs.groupby("group").apply(f)

## Static Visualizations in Python

Discussion and part of the next lecture will cover the plotnine package, an implementation of ggplot2 for Python. Unlike packages we've seen so far, plotnine is not included with Anaconda. To install the package:

* On Windows, run `conda install -c conda-forge plotnine` in an Anaconda Prompt (find it in the start menu)
* On MacOS or Linux, run `conda install -c conda-forge plotnine` in the Terminal

You may have to restart Jupyter after installing. To test your installation, try running

In [2]:
import plotnine

plotnine.__version__

'0.5.1'

## More About Packages and Modules

Which of the built-in modules are important?

Module      | Description
----------- | -----------
sys         | info about Python (version, etc)
pdb         | Python debugger
pathlib     | tools for file paths
collections | additional data structures
string      | string processing
re          | regular expressions
datetime    | date processing
urlparse    | tools for URLs
itertools   | tools for iterators
functools   | tools for functions

Python's built-in `math` and `statistics` modules are missing features we need for serious scientific computing, so we use the "SciPy Stack" instead.

The SciPy Stack is a collection of packages for scientific computing (marked with a `*` below). Most scientists working in Python use the SciPy Stack. The 3 most important packages in the stack are:

Package      | Description
------------ | -----------
numpy\*      | arrays, matrices, math/stat functions
scipy\*      | additional math/stat functions
pandas\*     | data frames

There are also several packages available for creating static plots. We'll see these soon:

Package      | Description
------------ | -----------
matplotlib\* | visualizations
seaborn      | "statistical" visualizations
plotnine     | ggplot2 for Python

Finally, there are many other packages we may use for specific statistical tasks. Some of these are:

Package      | Description
------------ | -----------
requests     | web (HTTP) requests
lxml         | web page parsing (XML & HTML)
beatifulsoup | web page parsing (HTML)
nltk         | natural language processing
spacy        | natural language processing
textblob     | natural language processing
statsmodels  | classical statistical models
scikit-learn | machine learning models
pillow       | image processing
scikit-image | image processing
opencv       | image processing